This notebook was made to collect the data in the UGC Fridge Portal through the STAC API.

In [17]:
import pystac_client as pc
import ipywidgets
import folium
from pystac import Item
from shapely.geometry import Point, mapping
from shapely.geometry.polygon import Polygon
from pyproj import Transformer
from shapely.ops import transform
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)




%matplotlib inline

In [10]:
cat = pc.Client.open("https://stac.pgc.umn.edu/api/v1/")
collection = cat.get_collection("arcticdem-strips-s2s041-2m")
print(collection)
collection = "arcticdem-strips-s2s041-2m"

<CollectionClient id=arcticdem-strips-s2s041-2m>


In [ ]:
# longitude is x, latitude is y

def getData(lat, lon):
    
    df = None
    
# this code generates a circle to query the data in
    # radius of circle in terms of meters
    radius = 10000

    # defines center of circle
    center = Point(lat, lon)
    
    # transforms from lat-lon to meters
    transform_to = Transformer.from_crs(4326, 3413, always_xy=True)
    projected = transform(transform_to.transform, center)
    
    # creates the circle with specified radius, higher resolution = smoother circle
    circle = projected.buffer(radius, resolution=500)
    
    # transforms from meters to lat-lon
    transform_back = Transformer.from_crs(3413, 4326, always_xy=True)
    projected = transform(transform_back.transform, circle)
    
    # saves the shape as a geojson
    geojson = shapely.to_geojson(projected)
    gdf = gpd.read_file(geojson)
    
    # filter out data
    for year in range(2008, 2025, 1):

        search = cat.search(
        collections = collection,
        intersects = geojson,
        datetime=f"{year}-07-15/{year}-07-25")
        
        try:
            if df is None:
                df = gpd.GeoDataFrame.from_features(search.item_collection().to_dict(), crs="epsg:4326").to_crs(3413)
                items = list(search.items())
                df["item"] = items
                # print(year)
            else: 
                # print(year)
                new_df = gpd.GeoDataFrame.from_features(search.item_collection().to_dict(), crs="epsg:4326").to_crs(3413)
                items = list(search.items())
                new_df["item"] = items
                df = pd.concat([df, new_df], ignore_index=True)

        except ValueError:
            # print(f"None: {year}")
            pass
        
    validity_mask = df["pgc:valid_area_percent"] > 0.9
    df = df[validity_mask]
    
    m = gdf.explore(name="AOI")
    m = df.explore(m=m, color='blue', name='Strips')
    folium.LayerControl().add_to(m)
    display(m)
    
    return df
    
    # data = []
    # for item in df["item"]:
    #     data.append(item.assets["dem"].href)
        
    # return data



In [36]:
data = getData(-42, 80) 
